## 🦜🔗 LangChain Agent mit einem SQL Toolkit


Zuerst enpacken wir unsere Demo Datenbank mit der wir gleich arbeiten werden.


In [ ]:
! [ ! -f northwind.db ] && bunzip2 -k northwind.db.bz2

#### Wir erstellen das SQL Toolkit und den Agenten


In [ ]:
from langchain_community.agent_toolkits.sql.base import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from helpers import llm

model = llm(temperature=0)
db = SQLDatabase.from_uri("sqlite:///northwind.db")
toolkit = SQLDatabaseToolkit(db=db, llm=model)
tools = toolkit.get_tools()

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain.schema import AIMessage, SystemMessage
from langchain_core.runnables import chain

system_message_prompt = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.
DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.
If the question does not seem related to the database, just return "I don't know" as the answer."""


@chain
def messages_modifier(messages):
    return [
        SystemMessage(system_message_prompt.format(dialect=toolkit.dialect, top_k=10)),
        messages[0],
        AIMessage(
            "I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables."
        ),
        *messages[1:],
    ]


agent_executor = create_react_agent(model, tools, messages_modifier=messages_modifier)

#### Looos....


In [ ]:
from langchain.schema import HumanMessage


input_1 = {"messages": [HumanMessage(content="Where do I find the orders?")]}
print(input_1["messages"][0])
for chunk in agent_executor.stream(input_1):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()

In [ ]:
input_2 = {"messages": [HumanMessage(content="Which Employee has the most orders?")]}
for chunk in agent_executor.stream(input_2):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()

#### Schauen wir mal, ob er das hier hinbekommt.


In [ ]:
input_3 = {
    "messages": [
        HumanMessage(
            content="Which Customer has had the Order with the highest total cost ever? What was the Order Id?"
        )
    ]
}
for chunk in agent_executor.stream(input_3):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()

#### Und noch einmal das batchen (async) demonstrieren.


In [ ]:
from typing import Dict


async def format_output(item: Dict) -> str:
    return [item.get("messages")[0].content, item.get("messages")[-1].content]


batcher = agent_executor | format_output

In [ ]:
result = await batcher.abatch([input_1, input_2, input_3])


for index, item in enumerate(result):
    print(f"Query {index+1}:")
    print(f"Question: {item[0]}")
    print(f"Answer: {item[1]}\n\n")

## ✅ Aufgabe

#### Ändert alle Namen, die ihr in der Datenbank findet auf "Little Bobbie Tables;" (https://xkcd.com/327/)


In [ ]:
print(db.get_table_info())

In [ ]:
your_input = {"messages": [HumanMessage(content="")]}
for chunk in agent_executor.stream(your_input):
    for state in chunk.values():
        for message in state["messages"]:
            message.pretty_print()